In [1]:
import cv2
import numpy as np
import sys
import keyboard
import time
import os
import glob

In [2]:
# Set the path to the images captured by the left and right cameras
pathL = "./right_calibration_images/"
pathR = "./left_calibration_images/"
 
# Termination criteria for refining the detected corners
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
 
objp = np.zeros((7*5,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)
 
img_ptsL = []
img_ptsR = []
obj_pts = []

for i in range(1,26):
    imgL = cv2.imread(pathL+"cal_right_img_%d.png"%i)
    imgR = cv2.imread(pathR+"cal_left_img_%d.png"%i)
    imgL_gray = cv2.imread(pathL+"cal_right_img_%d.png"%i,0)
    imgR_gray = cv2.imread(pathR+"cal_left_img_%d.png"%i,0)
     
    outputL = imgL.copy()
    outputR = imgR.copy()
     
    retR, cornersR =  cv2.findChessboardCorners(outputR,(7,5),None)
    retL, cornersL = cv2.findChessboardCorners(outputL,(7,5),None)
 
    if retR and retL:
        obj_pts.append(objp)
        cv2.cornerSubPix(imgR_gray,cornersR,(11,11),(-1,-1),criteria)
        cv2.cornerSubPix(imgL_gray,cornersL,(11,11),(-1,-1),criteria)
        cv2.drawChessboardCorners(outputR,(7,5),cornersR,retR)
        cv2.drawChessboardCorners(outputL,(7,5),cornersL,retL)
        cv2.imshow('cornersR',outputR)
        cv2.imshow('cornersL',outputL)
        cv2.waitKey(500)
             
        img_ptsL.append(cornersL)
        img_ptsR.append(cornersR)
 
cv2.destroyAllWindows() 
# Calibrating left camera
retL, mtxL, distL, rvecsL, tvecsL = cv2.calibrateCamera(obj_pts,img_ptsL,imgL_gray.shape[::-1],None,None)
hL,wL= imgL_gray.shape[:2]
new_mtxL, roiL= cv2.getOptimalNewCameraMatrix(mtxL,distL,(wL,hL),1,(wL,hL))
 
# Calibrating right camera
retR, mtxR, distR, rvecsR, tvecsR = cv2.calibrateCamera(obj_pts,img_ptsR,imgR_gray.shape[::-1],None,None)
hR,wR= imgR_gray.shape[:2]
new_mtxR, roiR= cv2.getOptimalNewCameraMatrix(mtxR,distR,(wR,hR),1,(wR,hR))

In [9]:
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC 
flags |= cv2.CALIB_SAME_FOCAL_LENGTH
# Here we fix the intrinsic camara matrixes so that only Rot, Trns, Emat and Fmat are calculated.
# Hence intrinsic parameters are the same 
 
criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
 
# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retS, new_mtxL, distL, new_mtxR, distR, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(obj_pts, img_ptsL, img_ptsR, new_mtxL, distL, new_mtxR, distR, imgL_gray.shape[::-1], criteria_stereo, flags)


In [10]:
rectify_scale= 1
rect_l, rect_r, proj_mat_l, proj_mat_r, Q, roiL, roiR= cv2.stereoRectify(new_mtxL, distL, new_mtxR, distR, imgL_gray.shape[::-1], Rot, Trns, rectify_scale,(0,0))


In [11]:
Left_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxL, distL, rect_l, proj_mat_l,
                                             imgL_gray.shape[::-1], cv2.CV_16SC2)
Right_Stereo_Map= cv2.initUndistortRectifyMap(new_mtxR, distR, rect_r, proj_mat_r,
                                              imgR_gray.shape[::-1], cv2.CV_16SC2)
 
print("Saving parameters......")
cv_file = cv2.FileStorage("improved_params2.xml", cv2.FILE_STORAGE_WRITE)
cv_file.write("Left_Stereo_Map_x",Left_Stereo_Map[0])
cv_file.write("Left_Stereo_Map_y",Left_Stereo_Map[1])
cv_file.write("Right_Stereo_Map_x",Right_Stereo_Map[0])
cv_file.write("Right_Stereo_Map_y",Right_Stereo_Map[1])
cv_file.release()
print("Done.")

Saving parameters......
Done.


In [12]:
cv2.imshow("Left image before rectification", imgL)
cv2.imshow("Right image before rectification", imgR)
 
Left_nice= cv2.remap(imgL,Left_Stereo_Map[0],Left_Stereo_Map[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
Right_nice= cv2.remap(imgR,Right_Stereo_Map[0],Right_Stereo_Map[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
 
cv2.imshow("Left image after rectification", Left_nice)
cv2.imshow("Right image after rectification", Right_nice)
cv2.waitKey(0)
 
out = Right_nice.copy()
out[:,:,0] = Right_nice[:,:,0]
out[:,:,1] = Right_nice[:,:,1]
out[:,:,2] = Left_nice[:,:,2]
 
cv2.imshow("Output image", out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
Left_Stereo_Map

(array([[[   231,    262],
         [   230,    262],
         [   229,    263],
         ...,
         [    93,    182],
         [    94,    179],
         [    96,    177]],
 
        [[   231,    262],
         [   230,    263],
         [   230,    263],
         ...,
         [    96,    177],
         [    98,    174],
         [   100,    172]],
 
        [[   231,    263],
         [   230,    263],
         [   230,    263],
         ...,
         [   100,    172],
         [   102,    169],
         [   104,    167]],
 
        ...,
 
        [[ 32767,  32767],
         [ 32767,  32767],
         [ 32767,  32767],
         ...,
         [-20298,  23450],
         [-20683,  23765],
         [-21075,  24084]],
 
        [[ 32767,  32767],
         [ 32767,  32767],
         [ 31933,  32767],
         ...,
         [-18197,  21041],
         [-18546,  21327],
         [-18901,  21616]],
 
        [[ 29808,  32767],
         [ 28873,  32767],
         [ 27966,  32767],
         